In [3]:
import pickle
import numpy as np
import pandas as pd
import xgboost as xgb
from xgboost import XGBClassifier
from multiprocessing import cpu_count

In [4]:
dataset = pd.read_csv('yoga/data.csv', names=['label']+['f'+str(i) for i in range(8)])

In [5]:
dataset['label'].unique()

array(['boundAngle', 'catCow', 'chairPose', 'cobra', 'downwardDog',
       'forwardBend', 'garland', 'halfForwardBend', 'highLunge', 'plank',
       'reverseWarrior', 'seatedSpinalTwist', 'threeLegDownwardDog',
       'tree', 'triangle'], dtype=object)

In [6]:
dataset

,label,f0,f1,f2,f3,f4,f5,f6,f7
0,boundAngle,0.17182845,0.31369594,2.7332203,2.5956583,1.4983823,1.6281252,0.55975443,0.44876382
1,boundAngle,0.29936314,0.36287594,2.479978,2.3593185,1.8934284,1.7367698,NaN,NaN
2,boundAngle,0.17372319,0.27472997,2.728917,2.5760143,1.4975827,1.7113018,0.5035153,0.504198
3,boundAngle,0.16526209,0.39916334,2.8196332,2.4437644,1.4055347,1.5379938,0.48966628,0.42372385
4,boundAngle,0.18356708,0.29015994,2.7066605,2.6916287,1.4934255,1.5314277,NaN,NaN
...,...,...,...,...,...,...,...,...,...
474,triangle,1.4245515,1.1607268,2.6008315,2.8499222,2.3884563,1.0140768,3.1169133,1.2575086
475,triangle,1.428369,1.1614816,2.9820669,2.8512406,2.3883684,1.0169783,3.1169932,1.2614369
476,triangle,1.432603,1.1623075,2.9825828,2.8526945,2.3882842,1.020195,3.1170733,1.2657948
477,triangle,1.4309102,1.1619793,2.604349,2.852115,2.3883157,1.0189095,3.1170416,1.2640527


In [7]:
poses = ['boundAngle', 'catCow', 'chairPose', 'cobra', 'downwardDog',
       'forwardBend', 'garland', 'halfForwardBend', 'highLunge', 'plank',
       'reverseWarrior', 'seatedSpinalTwist', 'threeLegDownwardDog',
       'tree', 'triangle']
print('poses:', len(poses))

poses: 15


In [8]:
dataset['label'] = dataset['label'].apply(lambda x: poses.index(x))

In [9]:
dataset

,label,f0,f1,f2,f3,f4,f5,f6,f7
0,0,0.17182845,0.31369594,2.7332203,2.5956583,1.4983823,1.6281252,0.55975443,0.44876382
1,0,0.29936314,0.36287594,2.479978,2.3593185,1.8934284,1.7367698,NaN,NaN
2,0,0.17372319,0.27472997,2.728917,2.5760143,1.4975827,1.7113018,0.5035153,0.504198
3,0,0.16526209,0.39916334,2.8196332,2.4437644,1.4055347,1.5379938,0.48966628,0.42372385
4,0,0.18356708,0.29015994,2.7066605,2.6916287,1.4934255,1.5314277,NaN,NaN
...,...,...,...,...,...,...,...,...,...
474,14,1.4245515,1.1607268,2.6008315,2.8499222,2.3884563,1.0140768,3.1169133,1.2575086
475,14,1.428369,1.1614816,2.9820669,2.8512406,2.3883684,1.0169783,3.1169932,1.2614369
476,14,1.432603,1.1623075,2.9825828,2.8526945,2.3882842,1.020195,3.1170733,1.2657948
477,14,1.4309102,1.1619793,2.604349,2.852115,2.3883157,1.0189095,3.1170416,1.2640527


In [10]:
X = dataset[['f'+str(i) for i in range(8)]].values  
y = dataset['label'].values

In [39]:
class_names = list(set(y))
num_class = len(class_names)
cores = cpu_count()

clf = XGBClassifier(max_depth=6, 
                    learning_rate=0.01, 
                    n_estimators=300, 
                    objective='multi:softmax', 
                    n_jobs=cores, 
                    num_class=num_class)

In [40]:
clf.fit(X, y, verbose=1)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.01, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=6, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=300, n_jobs=32, num_class=15, num_parallel_tree=None,
              objective='multi:softmax', ...)

In [41]:
filename = 'yoga/helpers/model/model_xgb.pkl'
pickle.dump(clf, open(filename, 'wb'))

In [42]:
y_pred = clf.predict(X)

In [43]:
from sklearn.metrics import accuracy_score, confusion_matrix

In [44]:
accuracy_score(y, y_pred)

1.0

In [45]:
confusion_matrix(y, y_pred)

array([[38,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0, 44,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0, 26,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0, 38,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0, 38,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0, 29,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0, 41,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0, 24,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0, 35,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0, 26,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 26,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 35,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 26,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 23,  0],
       [ 0,  0,  0,  0,  0,  0,  0